<a href="https://colab.research.google.com/github/shelvi31/Natural-Language-Processing/blob/main/Recognizing_Sentiment_Sarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [110]:
#importing
import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential


In [111]:
#Building classifier to Recognize sentiment and text -- Training a model to recognize sentiment in text 


In [112]:
#About Dataset

# Each record consists of three attributes:

# is_sarcastic: 1 if the record is sarcastic otherwise 0
# headline: the headline of the news article
# article_link: link to the original news article. Useful in collecting supplementary data

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [113]:
import json
import pandas as pd


import json
datastore = [json.loads(line) for line in open("/content/sample_data/Sarcasm_Headlines_Dataset_v2.json", 'r')]    #need to upload this file everytime with new runtime

sentences = []
labels = []

for item in datastore:
  sentences.append(item["headline"])
  labels.append(item["is_sarcastic"])

In [114]:
#splitting data for training and testing

training_size = 20000
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]

training_labels = labels[0:training_size]
testing_labels = sentences[training_size:]

In [115]:
vocab_size = 10000
embedding_dim = 16
max_length = 100

In [116]:
#lets do pre processing on the text

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


tokenizer = Tokenizer(num_words=10000,oov_token="<OOV>")   #alloting tokens to words

#asking tokenizer to go to text and fix them in sentences     
tokenizer.fit_on_texts(training_sentences)
  
word_index = tokenizer.word_index


#TRAINING
training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding="post", truncating='post')

#TESTING
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding="post", truncating="post")

In [117]:
# Need this block to get it to work with TensorFlow 2.x

import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [118]:
#Here is the neural network : for more clarity on concept : https://www.youtube.com/watch?v=Y_hzMnRXjhI&list=PLQY2H8rRoyvzDbLUZkbudP-MFQZwNmU4S&index=3
from keras.layers import Embedding

model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length = max_length),  #Embedding : Direction of each word (sarcastic or not) would be learnt epoch by epoch
        tf.keras.layers.GlobalAveragePooling1D(),   #Pool with global avg pooling : i.e adding vectors (direction of words)
#Feeding it to a neural network
        tf.keras.layers.Dense(24,activation="relu"),
        tf.keras.layers.Dense(1,activation="sigmoid")
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [119]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 100, 16)           160000    
_________________________________________________________________
global_average_pooling1d_13  (None, 16)                0         
_________________________________________________________________
dense_26 (Dense)             (None, 24)                408       
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 25        
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________


In [128]:
num_epochs = 10

model.fit(training_padded, training_labels, epochs=20, verbose=2)

Epoch 1/20
625/625 - 2s - loss: 0.0233 - accuracy: 0.9941
Epoch 2/20
625/625 - 2s - loss: 0.0230 - accuracy: 0.9944
Epoch 3/20
625/625 - 2s - loss: 0.0207 - accuracy: 0.9952
Epoch 4/20
625/625 - 2s - loss: 0.0174 - accuracy: 0.9959
Epoch 5/20
625/625 - 2s - loss: 0.0190 - accuracy: 0.9952
Epoch 6/20
625/625 - 2s - loss: 0.0168 - accuracy: 0.9964
Epoch 7/20
625/625 - 2s - loss: 0.0169 - accuracy: 0.9956
Epoch 8/20
625/625 - 2s - loss: 0.0135 - accuracy: 0.9969
Epoch 9/20
625/625 - 2s - loss: 0.0145 - accuracy: 0.9962
Epoch 10/20
625/625 - 2s - loss: 0.0128 - accuracy: 0.9969
Epoch 11/20
625/625 - 2s - loss: 0.0133 - accuracy: 0.9965
Epoch 12/20
625/625 - 2s - loss: 0.0109 - accuracy: 0.9973
Epoch 13/20
625/625 - 2s - loss: 0.0094 - accuracy: 0.9979
Epoch 14/20
625/625 - 2s - loss: 0.0093 - accuracy: 0.9979
Epoch 15/20
625/625 - 2s - loss: 0.0107 - accuracy: 0.9973
Epoch 16/20
625/625 - 2s - loss: 0.0091 - accuracy: 0.9977
Epoch 17/20
625/625 - 2s - loss: 0.0097 - accuracy: 0.9971
Epoch 

In [129]:
sentence = ["granny starting to fear spiders in the garden might be real", 
            "game of thrones season finale showing this sunday night"]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding="post", truncating="post")
print(model.predict(padded))

[[9.8571622e-01]
 [2.8493918e-08]]
